Simulation Code for We-DRIVE_U

In [31]:
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import copy

In [32]:
class American_put_option():
    def __init__(self, p0, d, seed=1):
        np.random.seed(seed)
        self.lower = 60
        self.upper = 160
        self.p0 = p0
        self.A = [0, 1] # a=0 is not exercising, a=1 is exercising
        self.K = 100
        self.epsilon = 5
        self.c = [1.02, 0.98]
        self.s0 = np.random.uniform(self.K - self.epsilon, self.K + self.epsilon, 1)
        self.d = d
        self.Delta = (self.upper - self.lower) / d
        self.anchor = [80 + x * self.Delta for x in range(d)]

    def reset(self):
        self.S = [self.s0]
        self.A = [] # save the action history
        self.R = [] # save the reward history
        self.h = 0 # reset the step to 0
        self.feature = [] # save the feature trajectory with respect to a=0
        self.current_state = self.s0  # reset the current state to initial state

    def phi(self, s):
        phi = [max(0, (1 - np.abs(s - si) / self.Delta)[0]) for si in self.anchor]    
        return np.array(phi)
    
    def add_state(self, s):
        self.S.append(s)
        self.current_state = s

    def update_state(self):
        idx = np.random.choice([0, 1], p = [self.p0, 1 - self.p0])
        sprime = self.current_state * self.c[idx]
        return sprime
    
    def next_state(self, a):
        if a == 1:
            next_state = np.array([99999])
            self.add_state(next_state)
            return next_state
        else:
            next_state = self.update_state()
            # next_state = max(self.lower, next_state)
            # next_state = min(self.upper, next_state)
            self.add_state(next_state)
            return next_state
    
    def generate_reward(self, a):
        if a == 0:
            reward = 0
            self.R.append(reward)
            return reward
        else:
            reward = max(0, self.K - self.current_state)
            self.R.append(reward)
            return reward
    
    def step(self, a):
        self.A.append(a)
        phi = self.phi(self.current_state)
        self.feature.append(phi)
        self.generate_reward(a)
        self.next_state(a)
        self.h += 1


In [ ]:
class LSVI_UCB():
    def __init__(self, H, beta, lam, d):
        self.lam = lam
        self.beta = beta
        self.H = H
        self.K = 100
        self.d = d
        self.w = [np.zeros(self.d) for _ in range(self.H)]
        self.Lambda = [self.lam * np.diag(np.ones(self.d)) for _ in range(self.H)]

    def get_action(self, phi, h, current_state):
        Q_exercising = max(0, self.K - current_state)
        Q_not_exercising = self.get_Q_func(phi, h)
        Q = [Q_not_exercising, Q_exercising]
        return np.argmax(Q)

    def get_Q_func(self, phi, h):
        Lambda_h_inverse = np.linalg.inv(self.Lambda[h])
        Q_h = np.min([(self.w[h] @ phi + self.beta * np.sqrt(phi @ Lambda_h_inverse @ phi)), self.H])
        return Q_h 
    
    def update_Q(self, history):
        # Backward induction
        self.w = [None for _ in range(self.H)] # initialize weights w
        for h in range(self.H-1, -1, -1):
            # update Lambda_h
            phi_h = history['phi'][-1][h]
            self.Lambda[h] += np.outer(phi_h, phi_h)
            # update w_h 
            Lambda_h_inverse = np.linalg.inv(self.Lambda[h])
            w_h = np.zeros(self.d)
            if h == self.H - 1:
                self.w[h] = w_h
            else:
                for tau in range(history['k']):
                    phi_tau_h = history['phi'][tau][h]
                    phi_tau_h_plus_one = history['phi'][tau][h+1]
                    s_tau_h_plus_one = history['state'][tau][h+1]
                    r_tau_h = history['r'][tau][h]
                    Q_tau_h_plus_one = [self.get_Q_func(phi_tau_h_plus_one, h + 1), max(0, self.K - s_tau_h_plus_one)]
                    V_tau_h_plus_one = np.max(Q_tau_h_plus_one)
                    w_h += Lambda_h_inverse @ (phi_tau_h * (r_tau_h + V_tau_h_plus_one))
            self.w[h] = w_h

class DR_LSVI_UCB():
    def __init__(self, d, beta, H, lam, rho):
        self.lam = lam
        self.d = d
        self.H = H
        self.K = 100
        self.beta = beta
        self.w = [np.zeros(self.d) for _ in range(self.H)]
        self.Lambda = [self.lam * np.diag(np.ones(self.d)) for _ in range(self.H)]
        self.rho = rho

    def get_action(self, phi, h, current_state):
        Q_exercising = max(0, self.K - current_state)
        Q_not_exercising = self.get_Q_func(phi, h)
        Q = [Q_not_exercising, Q_exercising]
        return np.argmax(Q)
    
    def get_Q_func(self, phi, h):
        Lambda_h_inverse = np.linalg.inv(self.Lambda[h])
        bonus =  self.beta * np.sqrt(phi @ np.diag(np.diagonal(Lambda_h_inverse)) @ phi)
        Q_h = np.min([(self.w[h] @ phi + bonus), self.H - h])
        return Q_h 
    
    def get_nu_h(self, history, h, rho):
        Lambda_h_inverse = np.linalg.inv(self.Lambda[h])
        nu_h = np.zeros(self.d)  
        Phi_h = np.zeros((0,self.d)) 
        V_h_plus_one = np.zeros(0)
        for tau in range(history['k']): 
            phi_tau_h = history['phi'][tau][h]
            Phi_h = np.vstack((Phi_h, phi_tau_h))
            phi_tau_h_plus_one = history['phi'][tau][h+1]
            s_tau_h_plus_one = history['state'][tau][h+1]
            Q_tau_h_plus_one = [self.get_Q_func(phi_tau_h_plus_one, h + 1), max(0, (self.K - s_tau_h_plus_one))]
            V_tau_h_plus_one = np.max(Q_tau_h_plus_one)
            V_h_plus_one = np.hstack((V_h_plus_one, V_tau_h_plus_one))
        for i in range(self.d):
            #print(i)
            def z_alpha_i(alpha):
                # compact formular for z
                z = Lambda_h_inverse @ Phi_h.T @ np.minimum(V_h_plus_one, alpha)
                return - z[i] + rho * alpha
            result =  minimize(z_alpha_i, self.H/2, method='Nelder-Mead', bounds=[(0,self.H)])
            nu_h[i] = - result.fun
            #print(result.x)
        return nu_h
    
    def update_Q(self, history): 
        # Backward induction
        self.w = [None for _ in range(self.H)] # initialize weights w
        for h in range(self.H-1, -1, -1):
            # update Lambda_h
            phi_h = history['phi'][-1][h]
            self.Lambda[h] += np.outer(phi_h, phi_h)
            # update w_h 
            if h == self.H - 1:
                w_h = np.zeros(self.d)
            else:
                nu_h = self.get_nu_h(history, h, rho=self.rho)
                w_h = nu_h
            self.w[h] = w_h

class VA_DR_LSVI_UCB():
    def __init__(self, d, beta, H, lam, rho):
        self.lam = lam
        self.d = d
        self.H = H
        self.K = 100
        self.beta = beta
        self.w = [np.zeros(self.d) for _ in range(self.H)]
        self.Lambda = [self.lam * np.diag(np.ones(self.d)) for _ in range(self.H)]
        self.Lambda_VA = [self.lam * np.diag(np.ones(self.d)) for _ in range(self.H)]
        self.Lambda_VA_last = [self.lam * np.diag(np.ones(self.d)) for _ in range(self.H)]
        self.rho = rho
        self.variance = {}
        self.switch = 0

    def get_action(self, phi, h, current_state):
        Q_exercising = max(0, self.K - current_state)
        Q_not_exercising = self.get_Q_func(phi, h)
        Q = [Q_not_exercising, Q_exercising]
        return np.argmax(Q)

    def get_variance_coefficient(self, h, history):
        Lambda_h_inverse = np.linalg.inv(self.Lambda[h])
        z1_h = np.zeros(self.d)
        z2_h = np.zeros(self.d)
        Phi_h = np.zeros((0,self.d))
        V_h_plus_one = np.zeros(0)
        for tau in range(history['k']):
            phi_tau_h = history['phi'][tau][h]
            Phi_h = np.vstack((Phi_h, phi_tau_h))
            phi_tau_h_plus_one = history['phi'][tau][h+1]
            s_tau_h_plus_one = history['state'][tau][h+1]
            Q_tau_h_plus_one = [self.get_Q_func(phi_tau_h_plus_one, h + 1), max(0, (self.K - s_tau_h_plus_one))]
            V_tau_h_plus_one = np.max(Q_tau_h_plus_one)
            V_h_plus_one = np.hstack((V_h_plus_one, V_tau_h_plus_one))

        z1_h = Lambda_h_inverse @ Phi_h.T @ V_h_plus_one
        z2_h = Lambda_h_inverse @ Phi_h.T @ V_h_plus_one**2
        return z1_h, z2_h

    def estimated_variance(self, phi, z1_h, z2_h):
        second_order_term = np.min([np.max([0, np.dot(phi, z2_h)]), self.H**2])
        first_order_term = np.min([np.max([0, np.dot(phi, z1_h)]), self.H])
        sigma_square = np.max([1, second_order_term - first_order_term**2])
        return sigma_square

    def get_Q_func(self, phi, h):
        Lambda_h_inverse = np.linalg.inv(self.Lambda_VA[h])
        bonus =  self.beta * np.sqrt(phi @ np.diag(np.diagonal(Lambda_h_inverse)) @ phi)
        Q_h = np.min([(self.w[h] @ phi + bonus), self.H - h])
        return Q_h

    def get_nu_h(self, history, h, rho, variance):
        Lambda_h_inverse = np.linalg.inv(self.Lambda_VA[h])
        nu_h = np.zeros(self.d)
        Phi_h = np.zeros((0, self.d))
        V_h_plus_one = np.zeros(0)
        for tau in range(history['k']):
            phi_tau_h = history['phi'][tau][h]
            Phi_h = np.vstack((Phi_h, phi_tau_h))
            phi_tau_h_plus_one = history['phi'][tau][h+1]
            s_tau_h_plus_one = history['state'][tau][h+1]
            Q_tau_h_plus_one = [self.get_Q_func(phi_tau_h_plus_one, h + 1), max(0, (self.K - s_tau_h_plus_one))]
            V_tau_h_plus_one = np.max(Q_tau_h_plus_one)
            V_h_plus_one = np.hstack((V_h_plus_one, V_tau_h_plus_one))
        for i in range(self.d):
            def z_alpha_i(alpha):
                # compact formular for z
                z = Lambda_h_inverse @ Phi_h.T @ (np.minimum(V_h_plus_one, alpha) / variance)
                return - z[i] + rho * alpha
            result =  minimize(z_alpha_i, self.H/2, method='Nelder-Mead', bounds=[(0,self.H)])
            nu_h[i] = - result.fun
        return nu_h

    def update_covariance_matrix(self, history):
        # Update Lambda_h and Lambda_VA_h
        for h in range(self.H-1, -1, -1):
            # update Lambda_h
            phi_h = history['phi'][-1][h]
            self.Lambda[h] += np.outer(phi_h, phi_h)
            # update Lambda_VA_h
            if h == self.H-1:
                self.Lambda_VA[h] += np.outer(phi_h, phi_h)
            else:
                # calculate variance estimator
                z1_h, z2_h = self.get_variance_coefficient(h, history)
                self.variance[str(h)] = np.zeros(history['k'])
                for tau in range(history['k']):
                    feature_temp = history['phi'][tau][h]
                    variance_temp = self.estimated_variance(feature_temp, z1_h, z2_h)
                    self.variance[str(h)][tau] = variance_temp
                    self.Lambda_VA[h] += np.outer(feature_temp, feature_temp) / variance_temp

    def update_Q(self, history):
        # check if the criterion is met
        criterion = False
        for h in range(self.H):
            if np.linalg.det(self.Lambda_VA[h]) >= 2 * np.linalg.det(self.Lambda_VA_last[h]):
                criterion = True
                break

        # if criterion is met
        if criterion:     
            # number of switch + 1
            self.switch += 1

            # Backward induction
            self.w = [None for _ in range(self.H)]
            for h in range(self.H-1, -1, -1):
                # update Lambda_VA_last
                self.Lambda_VA_last[h] = copy.copy(self.Lambda_VA[h])

                # update w_h
                if h == self.H - 1:
                    w_h = np.zeros(self.d)
                else:
                    variance = self.variance[str(h)]
                    nu_h = self.get_nu_h(history, h, rho = self.rho, variance=variance)
                    w_h = nu_h
                self.w[h] = w_h
        # if criterion is not met
        else:
            pass

In [ ]:
def train_once(epoch, H, beta, lam, d, p0, seed):
    history = {'k': 0, 'phi':[], 'r':[], 'state':[]}
    env = American_put_option(p0=p0, d=d, seed=seed)
    agent = LSVI_UCB(H=H, beta=beta, lam=lam, d=d)
    for t in range(epoch):
        env.reset()
        for h in range(H):
            current_state = env.current_state
            phi = env.phi(current_state) 
            action = agent.get_action(phi, h, current_state)
            env.step(action)
        # log the trajectory
        history['phi'].append(env.feature)
        history['r'].append(env.R)
        history['state'].append(env.S)
        history['k'] += 1
        # update the agent
        agent.update_Q(history)
    return agent

def train_once_DR(epoch, H, beta, lam, d, p0, rho, seed):
    history = {'k': 0, 'phi':[], 'r':[], 'state':[]}
    env = American_put_option(p0=p0, d=d, seed=seed)
    DR_agent = DR_LSVI_UCB(H=H, beta=beta, lam=lam, d=d, rho=rho)
    # reward = 0
    # Reward = []
    for t in range(epoch):
        env.reset()
        for h in range(H):
            current_state = env.current_state
            phi = env.phi(current_state) 
            action = DR_agent.get_action(phi, h, current_state)
            env.step(action)
        # log the trajectory
        history['phi'].append(env.feature)
        history['r'].append(env.R)
        history['state'].append(env.S)
        history['k'] += 1
        # update the agent
        DR_agent.update_Q(history)
    return DR_agent

def train_once_DR_VA(epoch, H, beta, lam, d, p0, rho, seed):
    history = {'k': 0, 'phi':[], 'r':[], 'state':[]}
    
    env = American_put_option(p0=p0, d=d, seed=seed)
    VA_DR_agent = VA_DR_LSVI_UCB(H=H, beta=beta, lam=lam, d=d, rho=rho)
    for t in range(epoch):
        env.reset()
        for h in range(H):
            current_state = env.current_state
            phi = env.phi(current_state) 
            action = VA_DR_agent.get_action(phi, h, current_state)
            env.step(action)
        # log the trajectory
        history['phi'].append(env.feature)
        history['r'].append(env.R)
        history['state'].append(env.S)
        history['k'] += 1
        # update the agent
        VA_DR_agent.update_covariance_matrix(history)
        VA_DR_agent.update_Q(history)
    return VA_DR_agent

In [ ]:
T1 = 100
H = 10
beta = 1
lam = 0.1
rho = 0.15
p0 = 0.5
d = 20
replication = 10
agent_dic = {}
DR_agent_dic = {}
VA_DR_agent_dic = {}

# Train on the source domain
for rep in range(replication):
    agent = train_once(epoch=T1, H=H, beta=beta, lam=lam, d=d, p0=p0, seed=rep)
    DR_agent = train_once_DR(epoch=T1, H=H, beta=beta, lam=lam, d=d, p0=p0, rho=rho, seed=rep)
    VA_DR_agent = train_once_DR_VA(epoch=T1, H=H, beta=beta, lam=lam, d=d, p0=p0, rho=rho, seed=rep)
    agent_dic[str(rep)] = agent
    DR_agent_dic[str(rep)] = DR_agent
    VA_DR_agent_dic[str(rep)] = VA_DR_agent


# Test on the target domain
PROB = [x / 40 for x in range(6,35)]
T2 = 100
R_LSVI_UCB = []
R_DR_LSVI_UCB = []
R_VA_DR_LSVI_UCB = []
for p in PROB:
    REWARD = 0
    REWARD_DR = 0
    REWARD_DR_VA = 0
    for rep in range(replication):
        reward = 0
        reward_DR = 0
        reward_DR_VA = 0
        env_test = American_put_option(p, d, seed=rep)
        env_test_DR = American_put_option(p, d, seed=rep)
        env_test_DR_VA = American_put_option(p, d, seed=rep)
        agent = agent_dic[str(rep)]
        DR_agent = DR_agent_dic[str(rep)]
        VA_DR_agent = VA_DR_agent_dic[str(rep)]
        for t in range(T2):
            env_test.reset()
            env_test_DR.reset()
            env_test_DR_VA.reset()
            for h in range(H):
                current_state = env_test.current_state
                phi = env_test.phi(current_state)
                action = agent.get_action(phi, h, current_state)

                current_state_DR = env_test_DR.current_state
                phi_DR = env_test_DR.phi(current_state_DR)
                action_DR = DR_agent.get_action(phi_DR, h, current_state_DR)

                current_state_DR_VA = env_test_DR_VA.current_state
                phi_DR_VA = env_test_DR_VA.phi(current_state_DR_VA)
                action_DR_VA = VA_DR_agent.get_action(phi_DR_VA, h, current_state_DR_VA)

                env_test.step(action)
                env_test_DR.step(action_DR)
                env_test_DR_VA.step(action_DR_VA)
            reward += np.sum(env_test.R) / T2
            reward_DR += np.sum(env_test_DR.R) / T2
            reward_DR_VA += np.sum(env_test_DR_VA.R) / T2
            
        REWARD += reward / replication
        REWARD_DR += reward_DR / replication
        REWARD_DR_VA += reward_DR_VA / replication
    R_LSVI_UCB.append(REWARD)
    R_DR_LSVI_UCB.append(REWARD_DR)
    R_VA_DR_LSVI_UCB.append(REWARD_DR_VA)
plt.plot(PROB, R_LSVI_UCB, label = 'LSVI-UCB')
plt.plot(PROB, R_DR_LSVI_UCB, label = 'DR-LSVI-UCB')
plt.plot(PROB, R_VA_DR_LSVI_UCB, label = 'We-DRIVE-U')
plt.legend(fontsize=16)
plt.xlabel('Price-up Probability', size=16)
plt.ylabel('Average Reward', size=16)
plt.savefig(f'APO_{p0}_{d}_{rho}.pdf', dpi=1000, bbox_inches='tight', pad_inches=0.0)
